In [7]:
// Enable Deno Kernel to run this notebook


Notebook looks for enemies sprites in existing https://github.com/freedoom/freedoom commits and saves them in structured way with commit metadata (to identify authors and creation dates)

In [16]:
// Clone the repository to avoid rate limits and increase speed of parsing
const cloneCmd = new Deno.Command("git", {
  args: [
    "clone",
    "--bare",
    "https://github.com/freedoom/freedoom.git",
    "freedoom.git",
  ],
});

cloneCmd.outputSync();


{
  success: false,
  code: 128,
  signal: null,
  stdout: [Getter],
  stderr: [Getter]
}

In [ ]:
// Run with: deno run --allow-read --allow-write --allow-run scan_CYBR_deep.ts

import { TextLineStream } from "https://deno.land/std@0.224.0/streams/mod.ts";

const CONFIG = {
  // Path to your local bare repo
  repoPath: "freedoom.git",

  // Base URL for links
  githubUrl: "https://github.com/freedoom/freedoom",

  // Regex matches: CYBR + Frame(a-z) + Angle(0-8) + Extra(a8..) + Extension
  // Matches: sprites/CYBRa2a8.png
  fileRegex: /(?:^|[\\/])CYBR([a-z])(\d).*?\.(png|gif)$/i,

  outputFile: "scan_results_CYBR.json",
};

/**
 * Runs git ls-tree to get ALL matching files at a specific commit.
 * Now scans root "." to ensure we catch files anywhere.
 */
async function getSnapshotFiles(sha: string): Promise<string[]> {
  const cmd = new Deno.Command("git", {
    args: [
      "--git-dir",
      CONFIG.repoPath,
      "ls-tree",
      "-r", // Recursive
      "--name-only",
      sha,
    ],
    stdout: "piped",
  });

  const { stdout } = await cmd.output();
  const output = new TextDecoder().decode(stdout);

  return output
    .split("\n")
    .map((s) => s.trim())
    .filter((s) => s.length > 0 && CONFIG.fileRegex.test(s));
}

async function main() {
  // 1. Validate Repo
  try {
    await Deno.stat(CONFIG.repoPath);
    console.log(`📂 Using Repo: ${CONFIG.repoPath}`);
  } catch {
    console.error(`❌ Error: Repo path not found: '${CONFIG.repoPath}'`);
    Deno.exit(1);
  }

  // 2. Debug: Check if the specific example commit exists
  const exampleSha = "57246cae8f7901d4bc63072f9632685d1e3b507d";
  const checkCmd = new Deno.Command("git", {
    args: ["--git-dir", CONFIG.repoPath, "cat-file", "-t", exampleSha],
  });
  const { success } = await checkCmd.output();
  if (success) {
    console.log(
      `✅ Verified: Commit ${exampleSha.slice(0, 7)} exists in local repo.`,
    );
  } else {
    console.warn(
      `⚠️  Warning: Commit ${
        exampleSha.slice(0, 7)
      } (from your URL) NOT found in local repo.`,
    );
    console.warn(
      `   Results might differ from GitHub if your local repo is incomplete.`,
    );
  }

  // 3. Run Deep Scan (No folder filter)
  console.log("🚀 Starting Deep Scan (Scanning root)...");

  const cmd = new Deno.Command("git", {
    args: [
      "--git-dir",
      CONFIG.repoPath,
      "log",
      "--name-status",
      "--pretty=format:__COMMIT__|%H|%cd|%an|%s",
      "--date=iso-strict",
      // REMOVED "sprites" folder argument to force full scan
    ],
    stdout: "piped",
  });

  const process = cmd.spawn();
  const lines = process.stdout
    .pipeThrough(new TextDecoderStream())
    .pipeThrough(new TextLineStream());

  const results: any[] = [];
  let currentCommit: any = null;
  let commitCount = 0;
  let matchCount = 0;

  const finishCommit = async (commit: any) => {
    if (commit && commit.changesMap.size > 0) {
      // We found a commit that touched a CYBR file.
      // Now let's grab the full snapshot of CYBR files at this moment.
      const allPaths = await getSnapshotFiles(commit.sha);

      const mergedFiles = allPaths.map((path) => {
        const status = commit.changesMap.get(path) || "Existing";
        return {
          path: path,
          status: status,
          url: `${CONFIG.githubUrl}/blob/${commit.sha}/${path}`,
        };
      });

      mergedFiles.sort((a, b) => a.path.localeCompare(b.path));

      if (mergedFiles.length > 0) {
        commit.files = mergedFiles;
        delete commit.changesMap;
        commit.id = `${commit.sha}--${commit.date}`;

        console.log(
          `\r✅ MATCH: ${
            commit.sha.slice(0, 7)
          } | Files: ${mergedFiles.length} | "${
            commit.message.slice(0, 40)
          }..."`,
        );
        results.push(commit);
        matchCount++;
      }
    }
  };

  for await (const line of lines) {
    const trimmed = line.trim();
    if (!trimmed) continue;

    if (trimmed.startsWith("__COMMIT__|")) {
      await finishCommit(currentCommit);
      commitCount++;

      const parts = trimmed.split("|");
      currentCommit = {
        sha: parts[1],
        date: parts[2],
        author: parts[3],
        message: parts.slice(4).join("|"),
        changesMap: new Map<string, string>(),
      };

      if (commitCount % 500 === 0) {
        await Deno.stdout.write(
          new TextEncoder().encode(
            `\r⚡ Scanned ${commitCount} commits | Found ${matchCount} matches...`,
          ),
        );
      }
    } else if (currentCommit) {
      const parts = trimmed.split("\t");
      // Format: M    path/to/file.png
      if (parts.length >= 2) {
        const statusRaw = parts[0];
        const filename = parts[1];

        if (CONFIG.fileRegex.test(filename)) {
          // Status check: A=Added, M=Modified, etc.
          currentCommit.changesMap.set(filename, statusRaw.charAt(0));
        }
      }
    }
  }

  await finishCommit(currentCommit);

  console.log(`\n\n========================================`);
  console.log(`SCAN COMPLETE`);
  console.log(`Total Commits: ${commitCount}`);
  console.log(`Matches Found: ${results.length}`);

  if (results.length > 0) {
    const jsonOutput = JSON.stringify(results, null, 2);
    await Deno.writeTextFile(CONFIG.outputFile, jsonOutput);
    console.log(`\n💾 Saved to: ${Deno.cwd()}/${CONFIG.outputFile}`);
  } else {
    console.log("No matching files found.");
  }
}

main().catch(console.error);


📂 Using Repo: freedoom.git


Promise { <pending> }

✅ Verified: Commit 57246ca exists in local repo.
🚀 Starting Deep Scan (Scanning root)...
✅ MATCH: f3312e2 | Files: 44 | "sprites: address #1210; fix tripod offse..."
✅ MATCH: 86482d5 | Files: 44 | "sprites: make worm and tripod flippable...."
✅ MATCH: 57246ca | Files: 65 | "png: Map color 255 to color 133 (#1003)..."
✅ MATCH: 9c6c681 | Files: 65 | "Burn all GIFs: Convert everything to PNG..."
✅ MATCH: 0cc698a | Files: 65 | "de-symlinkify: urric..."
✅ MATCH: 5210aed | Files: 130 | "sprites: Tweak Urric's boss demon sprite..."
✅ MATCH: 9522dd9 | Files: 130 | "sprites: Fix flipped monster sprite...."
✅ MATCH: 785a32c | Files: 130 | "sprites: reduce height of cyberdemon cor..."
✅ MATCH: 88f258d | Files: 130 | "sprites: new cyberdemon..."
✅ MATCH: ba234f8 | Files: 68 | "fix cyb hurt placeholder sprites oh, for..."
✅ MATCH: 4b69173 | Files: 68 | "add cyberdemon placeholder..."
✅ MATCH: 27aca39 | Files: 65 | "Import sprites..."


SCAN COMPLETE
Total Commits: 3223
Matches Found: 12

💾 Saved to

In [ ]:
// Run with: deno run --allow-read --allow-write --allow-net --allow-run generate_animations_CYBR.ts

import { ensureDir } from "https://deno.land/std@0.224.0/fs/ensure_dir.ts";
import { join } from "https://deno.land/std@0.224.0/path/mod.ts";

const CONFIG = {
  inputFile: "scan_results_CYBR.json", // Make sure this matches your scan output
  outputDir: "webp_CYBR",
  tempDir: "temp_frames_CYBR",
  delay: 40, // 40 ticks = ~200ms per frame
  concurrency: 5,
};

// REGEX EXPLANATION:
// (?:^|[\\/]) -> Start of string OR a directory separator (ignores parent folders)
// CYBR        -> The sprite name prefix
// ([a-z])     -> Group 1: The Frame letter (A, B, C...)
// (\d)        -> Group 2: The primary Angle digit (0-8)
// .*?         -> Non-greedy match for extra chars (like 'a8' in CYBRa2a8)
// \.(png|gif) -> Extension
const SPRITE_REGEX = /(?:^|[\\/])CYBR([a-z])(\d).*?\.(png|gif)$/i;

interface FileEntry {
  path: string;
  status: string;
  url: string;
}

interface CommitEntry {
  id?: string; // Optional unique ID from the scan script
  sha: string;
  date: string;
  files: FileEntry[];
}

async function checkImageMagick() {
  try {
    const cmd = new Deno.Command("magick", { args: ["-version"] });
    const { success } = await cmd.output();
    if (!success) throw new Error();
  } catch {
    console.error("❌ Error: ImageMagick ('magick') not found.");
    Deno.exit(1);
  }
}

// --- Image Utils ---

async function getMaxDimensions(
  folderPath: string,
): Promise<{ w: number; h: number }> {
  // Use ImageMagick to get dimensions of all images in the folder
  const cmd = new Deno.Command("magick", {
    args: ["identify", "-format", "%w,%h\n", join(folderPath, "*")],
  });

  const { stdout } = await cmd.output();
  const output = new TextDecoder().decode(stdout).trim();

  let maxW = 0;
  let maxH = 0;

  for (const line of output.split("\n")) {
    const parts = line.split(",");
    if (parts.length < 2) continue;

    const w = parseInt(parts[0], 10);
    const h = parseInt(parts[1], 10);

    if (!isNaN(w) && w > maxW) maxW = w;
    if (!isNaN(h) && h > maxH) maxH = h;
  }

  return { w: maxW, h: maxH };
}

function getRawUrl(blobUrl: string): string {
  // Convert GitHub blob URL to raw content URL
  return blobUrl
    .replace("github.com", "raw.githubusercontent.com")
    .replace("/blob/", "/");
}

function isValidImageHeader(data: Uint8Array): boolean {
  if (data.length < 4) return false;
  // GIF Header (GIF8)
  if (data[0] === 0x47 && data[1] === 0x49 && data[2] === 0x46) return true;
  // PNG Header (‰PNG)
  if (
    data[0] === 0x89 && data[1] === 0x50 && data[2] === 0x4E && data[3] === 0x47
  ) return true;
  return false;
}

async function downloadRealImage(
  url: string,
  depth = 0,
): Promise<Uint8Array | null> {
  if (depth > 3) return null; // Prevent infinite redirects
  const res = await fetch(url);
  if (!res.ok) return null;
  const buffer = await res.arrayBuffer();
  const data = new Uint8Array(buffer);

  if (isValidImageHeader(data)) return data;

  // Handle LFS pointers or redirects disguised as text files
  try {
    const textContent = new TextDecoder().decode(data).trim();
    if (textContent.length < 500 && !textContent.includes("\0")) {
      const currentUrlObj = new URL(url);
      const targetUrl = new URL(textContent, new URL(".", currentUrlObj)).href;
      if (targetUrl !== url) return downloadRealImage(targetUrl, depth + 1);
    }
  } catch { /* ignore */ }
  return null;
}

function getAnimationSequence(files: FileEntry[]) {
  const frames = new Map<string, FileEntry>();

  for (const file of files) {
    const match = file.path.match(SPRITE_REGEX);
    if (!match) continue;

    // Group 1: Frame Letter (A, B...)
    // Group 2: Angle (1-8, 0)
    const letter = match[1].toUpperCase();
    const angle = parseInt(match[2], 10);

    if (frames.has(letter)) {
      const existing = frames.get(letter)!;
      const existingMatch = existing.path.match(SPRITE_REGEX)!;
      const existingAngle = parseInt(existingMatch[2], 10);

      // Logic: Prefer Angle 1 (Front View) or Angle 0 (Omnidirectional)
      // If we have a side view (not 1 or 0), and find a front view (1), replace it.
      if (existingAngle !== 1 && angle === 1) {
        frames.set(letter, file);
      } // If we have a side view, don't have a front view, but find an omni view (0), take it.
      else if (existingAngle !== 1 && existingAngle !== 0 && angle === 0) {
        frames.set(letter, file);
      }
    } else {
      frames.set(letter, file);
    }
  }

  // Sort frames alphabetically (A, B, C...)
  return Array.from(frames.keys()).sort().map((key) => frames.get(key)!);
}

// --- Main Processing ---

async function processCommit(commit: CommitEntry) {
  // Use the unique ID generated by scan script, or fallback to SHA.
  // Sanitize for filename safety.
  const uniqueId = commit.id || commit.sha;
  const safeId = uniqueId.replace(/[^a-z0-9-]/gi, "_").slice(0, 100);

  const sequence = getAnimationSequence(commit.files);

  if (sequence.length === 0) return;

  const commitTempDir = join(CONFIG.tempDir, safeId);
  await ensureDir(commitTempDir);

  let validFramesCount = 0;

  try {
    let index = 0;
    for (const file of sequence) {
      const rawUrl = getRawUrl(file.url);
      const ext = file.path.split(".").pop() || "png";
      const localName = `${String(index).padStart(3, "0")}.${ext}`;
      const localPath = join(commitTempDir, localName);

      const imageBuffer = await downloadRealImage(rawUrl);
      if (!imageBuffer) continue;

      await Deno.writeFile(localPath, imageBuffer);
      validFramesCount++;
      index++;
    }

    if (validFramesCount < 2) return;

    // 1. Calculate Dimensions (so we can center/extent properly)
    const { w, h } = await getMaxDimensions(commitTempDir);

    if (w === 0 || h === 0) {
      console.warn(`⚠️  Could not determine dimensions for ${safeId}`);
      return;
    }

    const outputFilename = `CYBR-${safeId}.webp`;
    const outputPath = join(CONFIG.outputDir, outputFilename);

    // 2. Build ImageMagick Command
    const magickArgs = [
      "-delay",
      String(CONFIG.delay),
      "-dispose",
      "2", // Clear frame before rendering next (prevents ghosting)
      "-background",
      "none",
      "-loop",
      "0",
      join(commitTempDir, "*"),

      // Cleanup: Remove Cyan background if present (common in Doom sprites)
      "-transparent",
      "cyan",

      // Alignment: Center at the bottom
      "+repage",
      "-gravity",
      "South",
      "-extent",
      `${w}x${h}`,

      // Upscaling: 400% Nearest Neighbor (Pixel Art style)
      "-sample",
      "400%",

      // Output Format: Lossless WebP
      "-define",
      "webp:lossless=true",
      outputPath,
    ];

    const cmd = new Deno.Command("magick", { args: magickArgs });
    const { success, stderr } = await cmd.output();

    if (!success) {
      throw new Error(new TextDecoder().decode(stderr));
    }

    console.log(
      `✅ Generated: ${outputFilename} (${validFramesCount} frames, Size: ${
        w * 4
      }x${h * 4})`,
    );
  } catch (err: any) {
    console.error(`❌ Error processing ${safeId}:`, err.message);
  } finally {
    // Cleanup temp files for this commit
    try {
      await Deno.remove(commitTempDir, { recursive: true });
    } catch { /* ignore */ }
  }
}

async function main() {
  await checkImageMagick();

  console.log(`📂 Reading ${CONFIG.inputFile}...`);
  try {
    const raw = await Deno.readTextFile(CONFIG.inputFile);
    const commits: CommitEntry[] = JSON.parse(raw);

    console.log(`📁 Ensuring output directory: ${CONFIG.outputDir}`);
    await ensureDir(CONFIG.outputDir);
    await ensureDir(CONFIG.tempDir);

    console.log(`🚀 Processing ${commits.length} commit entries...`);

    // Run in chunks to avoid overwhelming network/CPU
    for (let i = 0; i < commits.length; i += CONFIG.concurrency) {
      const chunk = commits.slice(i, i + CONFIG.concurrency);
      await Promise.all(chunk.map(processCommit));
    }

    // Final cleanup of temp directory
    try {
      await Deno.remove(CONFIG.tempDir, { recursive: true });
    } catch { /* ignore */ }

    console.log("\n✨ All Done!");
  } catch (e: any) {
    console.error("Error:", e.message);
  }
}

main().catch(console.error);


Promise { <pending> }

📂 Reading scan_results_CYBR.json...
📁 Ensuring output directory: webp_CYBR
🚀 Processing 12 commit entries...
✅ Generated: CYBR-86482d5a10f7a6dcb0a913d01b839cf54c240dce--2023-10-30T09_28_37-07_00.webp (16 frames, Size: 452x428)
✅ Generated: CYBR-57246cae8f7901d4bc63072f9632685d1e3b507d--2023-07-16T23_14_24-07_00.webp (16 frames, Size: 452x428)
✅ Generated: CYBR-0cc698ae1fa04b19e30605e5479666c788fd5d4c--2015-12-17T01_22_31-08_00.webp (16 frames, Size: 452x428)
✅ Generated: CYBR-9c6c68127672feee501d5e5cab9d5e15c3dbf8b3--2017-07-18T22_26_52-07_00.webp (16 frames, Size: 452x428)
✅ Generated: CYBR-f3312e202b1ddeee3a2a42d303a686716d4eca17--2023-12-05T11_24_19-08_00.webp (16 frames, Size: 452x428)
✅ Generated: CYBR-785a32ce458ee7dd0cea1e8b14cd7e69725c76d5--2011-11-14T18_48_46Z.webp (16 frames, Size: 472x472)
✅ Generated: CYBR-88f258d181620c19db1256e85d3bd8b8a3dc7ec7--2010-09-13T00_38_11-07_00.webp (16 frames, Size: 472x472)
✅ Generated: CYBR-5210aedb76fd4990f628d133d3122602675e5080--2014-09-27